# Convert Parquet to Delta Lake

In [2]:
import pyspark
from delta import *

In [3]:
builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

In [5]:
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [6]:
columns = ["language", "num_speakers"]
data = [("English", "1.5"), ("Mandarin", "1.1"), ("Hindi", "0.6")]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

## Unpartitioned Parquet to Delta Lake

In [7]:
df.write.format("parquet").save("tmp/lake1")

In [8]:
!tree tmp/lake1

tmp/lake1
├── _SUCCESS
├── part-00000-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet
├── part-00003-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet
├── part-00006-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet
└── part-00009-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet

0 directories, 5 files


In [9]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`tmp/lake1`")

In [10]:
!tree tmp/lake1

tmp/lake1
├── _SUCCESS
├── _delta_log
│   ├── 00000000000000000000.checkpoint.parquet
│   ├── 00000000000000000000.json
│   └── _last_checkpoint
├── part-00000-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet
├── part-00003-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet
├── part-00006-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet
└── part-00009-4d33bcc0-8608-44e7-9e8b-c40e5f2e757c-c000.snappy.parquet

1 directory, 8 files


## Partitioned Parquet to Delta Lake

In [11]:
df.write.partitionBy("language").format("parquet").save("tmp/lake2")

In [12]:
!tree tmp/lake2

tmp/lake2
├── _SUCCESS
├── language=English
│   └── part-00003-6a7e03f7-47fe-49d6-85e5-9419e7b6630d.c000.snappy.parquet
├── language=Hindi
│   └── part-00009-6a7e03f7-47fe-49d6-85e5-9419e7b6630d.c000.snappy.parquet
└── language=Mandarin
    └── part-00006-6a7e03f7-47fe-49d6-85e5-9419e7b6630d.c000.snappy.parquet

3 directories, 4 files


In [13]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`tmp/lake2`")

AnalysisException: Expecting 0 partition column(s): [], but found 1 partition column(s): [`language`] from parsing the file name: file:/Users/matthew.powers/Documents/code/my_apps/delta-examples/notebooks/pyspark/tmp/lake2/language=English/part-00003-6a7e03f7-47fe-49d6-85e5-9419e7b6630d.c000.snappy.parquet

In [48]:
!tree tmp/lake2

tmp/lake2
├── _SUCCESS
├── _delta_log
├── language=English
│   └── part-00003-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet
├── language=Hindi
│   └── part-00009-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet
└── language=Mandarin
    └── part-00006-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet

4 directories, 4 files


In [49]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`tmp/lake2`", "language STRING")

In [50]:
!tree tmp/lake2

tmp/lake2
├── _SUCCESS
├── _delta_log
│   ├── 00000000000000000000.checkpoint.parquet
│   ├── 00000000000000000000.json
│   └── _last_checkpoint
├── language=English
│   └── part-00003-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet
├── language=Hindi
│   └── part-00009-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet
└── language=Mandarin
    └── part-00006-fa662100-1eff-4609-a0dd-794b5eec991a.c000.snappy.parquet

4 directories, 7 files


## Convert Parquet Lake with Multiple Appends

In [ ]:
df.write.format("parquet").save("tmp/lake3")

In [51]:
columns = ["language", "num_speakers"]
data = [("Spanish", "0.5"), ("French", "0.3"), ("Arabic", "0.3")]
rdd = spark.sparkContext.parallelize(data)
df = rdd.toDF(columns)

In [52]:
df.write.mode("append").format("parquet").save("tmp/lake3")

In [53]:
deltaTable = DeltaTable.convertToDelta(spark, "parquet.`tmp/lake3`")

In [56]:
deltaTable.version(0).show()

AttributeError: 'DeltaTable' object has no attribute 'version'

## Convert Parquet Lake no statistics

In [16]:
df.write.format("parquet").saveAsTable("lake4")

In [20]:
spark.sql("CONVERT TO DELTA lake4 NO STATISTICS")

ParseException: 
Syntax error at or near 'CONVERT'(line 1, pos 0)

== SQL ==
CONVERT TO DELTA lake4 NO STATISTICS
^^^


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 52063)
Traceback (most recent call last):
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 360, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/socketserver.py", line 747, in __init__
    self.handle()
  File "/Users/matthew.powers/opt/miniconda3/envs/pyspark-330-delta-210/lib/python3.9/site-packages/pyspark/accumulators.py", line 281, in ha

## Cleanup

In [15]:
%rm -rf tmp